In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('/content/flipkart_product.csv', encoding='iso-8859-1', on_bad_lines='skip', lineterminator='\n')

#Data Cleaning


In [22]:
df=df.loc[:,['Rate','Summary\r']]

In [23]:
df.isnull().sum()
df.dropna(inplace=True)

In [24]:
df.head()

,Rate,Summary\r
0,5,Great cooler.. excellent air flow and for this...
1,5,Best budget 2 fit cooler. Nice cooling\r
2,3,The quality is good but the power of air is de...
3,1,Very bad product it's a only a fan\r
4,3,Ok ok product\r


In [25]:
df.rename(columns={'Summary\r':'Summary'},inplace=True)

In [26]:
set(df['Rate'])

{'1',
 '2',
 '3',
 '4',
 '5',
 'Bajaj DX 2 L/W Dry Iron',
 'Nova Plus Amaze NI 10 1100 W Dry Iron?¿èGrey & Turquoise)',
 'Pigeon Favourite Electric Kettle?¿è1.5 L, Silver, Black)',
 's'}

In [27]:
list_value=['0','1','2','3','4','5']
df.drop(df[~df['Rate'].isin(list_value)].index,inplace=True)

In [28]:
df.shape

(185545, 2)

In [29]:
for index in df.index:
    df.loc[index, 'Rate'] = 0 if int(df.loc[index, 'Rate']) <= 2 else 1

In [30]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [32]:
df['Summary']

0         Great cooler.. excellent air flow and for this...
1                  Best budget 2 fit cooler. Nice cooling\r
2         The quality is good but the power of air is de...
3                      Very bad product it's a only a fan\r
4                                           Ok ok product\r
                                ...                        
187430                                               Good\r
187431                                             Thanks\r
187432                                               Good\r
187433                                              Super\r
187434                                               Good\r
Name: Summary, Length: 185545, dtype: object

#Removing Stop words and Steming


In [34]:
corpus=[]
for i in df.index:
  review=re.sub('[^a-zA-z]',' ',df.loc[i,'Summary'])
  review=review.lower()
  review=review.split()
  # review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=[word for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)


In [36]:
# corpus

In [37]:
Y=df['Rate']

In [39]:
len(Y),len(corpus)

(185545, 185545)

#Split train-test Data

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,Y,test_size=0.20)

#Bag of words for Words Embeddings

In [44]:
## Create the Bag OF Words model
from sklearn.feature_extraction.text import CountVectorizer
## for Binary BOW enable binary=True
cv=CountVectorizer(max_features=2500,ngram_range=(1,2))

In [48]:
X_train=cv.fit_transform(X_train)
X_test=cv.transform(X_test)

In [49]:
X_train=X_train.toarray()
X_test=X_test.toarray()

In [53]:
len(X_train[0])

2500

In [55]:
cv.vocabulary_

{'good': 776,
 'buy': 237,
 'good buy': 787,
 'every': 565,
 'thing': 2203,
 'range': 1802,
 'small': 2026,
 'ok': 1410,
 'every thing': 567,
 'good range': 844,
 'product': 1625,
 'good product': 840,
 'best': 147,
 'market': 1216,
 'powerful': 1582,
 'motor': 1299,
 'dont': 509,
 'think': 2205,
 'twice': 2269,
 'best market': 155,
 'far': 621,
 'works': 2454,
 'fine': 652,
 'works fine': 2455,
 'bad': 99,
 'good bad': 779,
 'nice': 1341,
 'happy': 921,
 'liked': 1128,
 'shape': 1980,
 'elegant': 548,
 'look': 1152,
 'heat': 941,
 'cut': 410,
 'mins': 1260,
 'smart': 2031,
 'looking': 1157,
 'watch': 2374,
 'received': 1835,
 'product good': 1652,
 'bicycle': 174,
 'great': 880,
 'fitness': 663,
 'sir': 2013,
 'beautiful': 130,
 'according': 5,
 'price': 1594,
 'though': 2211,
 'much': 1308,
 'finished': 655,
 'piece': 1532,
 'love': 1173,
 'according price': 6,
 'nyc': 1402,
 'hai': 908,
 'cleaner': 313,
 'ok ok': 1412,
 'ball': 108,
 'also': 29,
 'delivery': 453,
 'delayed': 447,
 '

#Apply Naive Bayes

In [56]:
from sklearn.naive_bayes import MultinomialNB

In [68]:
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [69]:
spam_detect_model=MultinomialNB().fit(X_train,y_train)

In [70]:
y_pred=spam_detect_model.predict(X_test)

In [71]:
from sklearn.metrics import accuracy_score,classification_report

In [72]:
accuracy_score(y_test,y_pred)

0.9324153170389933

In [74]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      0.70      0.74      5086
           1       0.95      0.97      0.96     32023

    accuracy                           0.93     37109
   macro avg       0.87      0.83      0.85     37109
weighted avg       0.93      0.93      0.93     37109

